# Notebook 1: Incremental Inference Logging

**Objective:** Load the black-box sentiment classifier, process a subset of the IMDB dataset, and log the model's prediction probabilities incrementally (token by token) for each review. These logged trajectories will be the observation sequences for training our HMM surrogate.

In [ ]:
import sys
import os
import numpy as np
import torch

# Add src directory to Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config import (
    MODEL_NAME, DATASET_NAME, 
    NUM_TRAIN_SAMPLES, MAX_TOKENS, LOG_FILE_PATH, DEVICE
)
from src.data_utils import get_tokenizer, load_imdb_data, preprocess_data_for_inference_logging
from src.black_box_model import BlackBoxSentimentClassifier, log_inference_trajectories

print(f"Using device: {DEVICE}")

## 1. Initialize Black-Box Model and Tokenizer

In [ ]:
bb_model = BlackBoxSentimentClassifier(model_name=MODEL_NAME, device=DEVICE)
tokenizer = bb_model.tokenizer 

## 2. Load and Preprocess Data

In [ ]:
imdb_train_raw = load_imdb_data(split='train', num_samples=NUM_TRAIN_SAMPLES, shuffle=True)
print(f"Loaded {len(imdb_train_raw)} raw training samples.")

processed_train_data = preprocess_data_for_inference_logging(imdb_train_raw, tokenizer)
print(f"Processed {len(processed_train_data)} samples for HMM training set generation.")

## 3. Log Inference Trajectories

In [ ]:
train_trajectories = log_inference_trajectories(processed_train_data, bb_model, max_len=MAX_TOKENS)

train_trajectories = [t for t in train_trajectories if t.shape[0] > 0]

print(f"Generated {len(train_trajectories)} trajectories for HMM training.")
if train_trajectories:
    print(f"Example trajectory 0 shape: {train_trajectories[0].shape}")
    print(f"Example trajectory 0 first 3 steps:\n{train_trajectories[0][:3]}")

## 4. Save Logged Trajectories

We save the list of numpy arrays. `np.savez_compressed` is suitable for this.

In [ ]:
if not os.path.exists('data'):
    os.makedirs('data')

if train_trajectories:
    np.savez_compressed(LOG_FILE_PATH, *train_trajectories) # Use * to save as separate arrays arr_0, arr_1, ...
    print(f"Saved {len(train_trajectories)} trajectories to {LOG_FILE_PATH}")
else:
    print("No trajectories to save.")

### Verify Save/Load (Optional)

In [ ]:
if train_trajectories:
    loaded_data = np.load(LOG_FILE_PATH, allow_pickle=True) # allow_pickle might be needed if arrays are objects
    loaded_trajectories = [loaded_data[f'arr_{i}'] for i in range(len(loaded_data.files))]
    print(f"Loaded back {len(loaded_trajectories)} trajectories.")
    assert len(loaded_trajectories) == len(train_trajectories)
    if loaded_trajectories:
        assert np.allclose(loaded_trajectories[0], train_trajectories[0])
        print("Verification successful.")